In [ ]:
!pip install --upgrade transformers

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `model-1` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `model-1`


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import subprocess
import tempfile
import json
import gc
import time

In [ ]:
import argparse
import os
import subprocess
import tempfile
import gc
import json
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- Explanation ---
# This script implements an AI-powered code assistant agent. It takes a user's request in natural language,
# uses a pre-trained Llama model (from the transformers library) to generate Python code implementing the requested feature,
# and then attempts to test the generated code to ensure it doesn't introduce errors. The agent incorporates a retry
# mechanism for code generation, expects structured JSON responses from the LLM, and includes memory
# management techniques (lazy model loading and explicit garbage collection).
# --- End Explanation ---

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = None
model = None

def load_model():
    """Loads the tokenizer and model if they are not already loaded."""
    global tokenizer, model
    if tokenizer is None or model is None:
        print(f"Loading model '{MODEL_NAME}'...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
        print("Model loaded.")

def unload_model():
    """Unloads the tokenizer and model to free up memory."""
    global tokenizer, model
    if tokenizer:
        del tokenizer
        tokenizer = None
    if model:
        del model
        model = None
    gc.collect()
    print("Llama model resources released.")

def test_code_snippet(code_snippet, timeout=10):
    """Attempts to execute the provided code snippet and captures errors."""
    try:
        with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=True) as tmp_file:
            tmp_file.write(code_snippet)
            tmp_file.flush()
            result = subprocess.run(['python', tmp_file.name], capture_output=True, text=True, timeout=timeout, check=False)  # Added check=False
            if result.stderr:
                return False, result.stderr
            return True, "Code executed without errors."
    except subprocess.TimeoutExpired:
        return False, f"Code execution timed out after {timeout} seconds."
    except FileNotFoundError:
        return False, "Python interpreter not found. Ensure Python is installed and in your PATH."
    except Exception as e:
        return False, f"An unexpected error occurred during code execution: {e}"

def test_code_with_existing(file_path, timeout=30):
    """Attempts to run existing tests in the project using pytest."""
    try:
        result = subprocess.run(['pytest', file_path], capture_output=True, text=True, timeout=timeout, check=False) # Added check=False
        if result.returncode == 0:
            return True, "Existing tests passed."
        else:
            return False, f"Existing tests failed with return code {result.returncode}:\n{result.stderr}\n{result.stdout}"
    except FileNotFoundError:
        return True, "Pytest not found. Skipping existing tests."
    except subprocess.TimeoutExpired:
        return False, f"Existing tests timed out after {timeout} seconds."
    except Exception as e:
        return False, f"An unexpected error occurred while running existing tests: {e}"

def implement_feature(file_content, user_request, num_retries=3):
    load_model()  # Load model before generating
    for attempt in range(num_retries):
        try:
            implementation_prompt = (
                "You are a helpful coding assistant. Please analyze the following Python code and implement the user's request.\n"
                "Respond with a JSON object containing 'code' (the complete modified code) and optionally 'explanation'.\n"
                "Ensure the implemented code is functional and integrates seamlessly. Maintain the original style where appropriate.\n\n"
                f"Existing Code:\n{file_content}\n\n"
                f"User Request: {user_request}"
            )
            inputs = tokenizer(implementation_prompt, return_tensors="pt")
            generate_ids = model.generate(inputs.input_ids, max_new_tokens=1200)
            response_text = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]

            try:
                response_json = json.loads(response_text)
                modified_code = response_json.get("code")
                explanation = response_json.get("explanation", "")
                print("\nProposed Implementation:")
                print(modified_code)
                if explanation:
                    print("\nExplanation:")
                    print(explanation)
                return modified_code
            except json.JSONDecodeError as e:
                print(f"Warning: Could not decode LLM response as JSON (attempt {attempt + 1}/{num_retries}). Error: {e}. Retrying...")
                if attempt == num_retries - 1:
                    print(f"Last attempt failed. Raw response:\n{response_text}")
                continue

        except Exception as e:
            print(f"Error during code generation (attempt {attempt + 1}/{num_retries}): {e}")
            if attempt < num_retries - 1:
                print("Retrying...")
                time.sleep(1) # Add a small delay before retrying
            else:
                print("All retry attempts failed.")
                return None
        finally:
            gc.collect() # Explicit garbage collection after each attempt

    return None

def assist_user(file_path):
    try:
        with open(file_path, 'r') as file:
            original_content = file.read()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return
    except Exception as e:
        print(f"Error reading file '{file_path}': {e}")
        return

    user_request = input("Describe the feature you want to implement: ")
    if not user_request.strip():
        print("No feature request provided.")
        return

    print("\nEngaging the AI...")
    modified_code = implement_feature(original_content, user_request)

    if modified_code:
        print("\nVerifying the implementation with existing knowledge...")
        tests_passed_existing, test_output_existing = test_code_with_existing(file_path)
        if tests_passed_existing:
            print(f"Existing codebase remains stable: {test_output_existing}")
        else:
            print(f"Caution! Existing codebase shows instability:\n{test_output_existing}")

        print("\nPerforming deep verification of the new implementation...")
        is_valid_snippet, test_output_snippet = test_code_snippet(modified_code)
        if is_valid_snippet:
            print(f"New implementation appears sound: {test_output_snippet}")
            user_response = input("Apply these enhancements? (y/n): ").strip().lower()
            if user_response == 'y':
                try:
                    with open(file_path, 'w') as file:
                        file.write(modified_code)
                    print(f"Enhancements woven into {file_path}.")
                except Exception as e:
                    print(f"Error writing to file '{file_path}': {e}")
            else:
                print("Enhancements held back.")
        else:
            print(f"Warning: New implementation shows instability:\n{test_output_snippet}")
            print("Implementation discarded due to potential issues.")
    else:
        print("Could not generate a valid implementation after multiple attempts.")

    unload_model() # Unload model after use

def main():
    parser = argparse.ArgumentParser(description="AI-Assisted Code Implementation and Verification")
    parser.add_argument("source_file", type=str, help="Source file to modify")
    args = parser.parse_args()

    if not os.path.exists(args.source_file):
        print(f"Error: File '{args.source_file}' not found.")
        return

    assist_user(args.source_file)

if __name__ == "__main__":
    main()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Engaging the AI...
